In [1]:
from unstructured.partition.auto import partition

pdf_path = 'pdf_2.pdf'

# Extract elements using Unstructured.io
unstructured_elements = partition(
    filename=pdf_path, 
    chunking_strategy="by_title",   # Tách theo heading/title nếu có thể
    languages=["vie"]
)

# Concatenate the text content of all extracted elements
unstructured_text = "\n".join([str(el) for el in unstructured_elements])

print("\n--- Unstructured.io Elements (first 5) ---")
for i, element in enumerate(unstructured_elements[:5]):
    print(f"Element Type: {type(element)}")
    print(f"Text: {element}\n")


--- Unstructured.io Elements (first 5) ---
Element Type: <class 'unstructured.documents.elements.CompositeElement'>
Text: TÀI LIỆU TẬP HUẤN TOT VỀ KỸ THUẬT SẢN XUẤT HỒ TIÊU BỀN VỮNG

1

TÀI LIỆU TẬP HUẤN TOT VỀ KỸ THUẬT SẢN XUẤT HỒ TIÊU BỀN VỮNG

3

MỤC LỤC

PHẦN I: HƯỚNG DẪN KỸ THUẬT SẢN XUẤT HỒ TIÊU

BÀI 1: GIỐNG VÀ KỸ THUẬT TRỒNG MỚI, TÁI CANH HỒ TIÊU 1. Một số giống tiêu phổ biến ở Việt Nam 1.1. Giống tiêu Vĩnh Linh

1.2. Giống Lada Belangtoeng

1.3. Các giống tiêu sẻ

1.4. Tiêu Ấn Độ

1.5. Giống tiêu Trâu

Element Type: <class 'unstructured.documents.elements.CompositeElement'>
Text: 1.6. Giống Phú quốc

2. Kỹ thuật nhân giống hồ tiêu: 2.1. Thiết kế vườn ươm giống tiêu 2.2. Chuẩn bị giá thể ươm

2.3. Đóng bầu

2.4. Chọn vườn tiêu lấy hom

2.5. Xử lý hom tiêu

2.6. Ươm, chăm sóc cây con

2.7. Tiêu chuẩn cây tiêu giống trước khi xuất vườn.

3. Kỹ thuật trồng mới hồ tiêu 3.1. Chọn đất trồng 3.2. Thiết kế lô, mật độ

3.3. Các loại trụ trồng tiêu

3.4. Thiết kế hệ thống cây chắn gi

In [2]:
import re

# Define the regex pattern for identifying titles
# It looks for lines starting with:
# - One or more Roman numerals followed by a dot and space (e.g., I., II.)
# - One or more groups of digits followed by a dot, ending with a dot and space (e.g., 1., 1.1., 2.3.4.)
# - A lowercase letter followed by a closing parenthesis or a dot, and a space (e.g., a), b., c.)
# - The keywords "BÀI", "CHƯƠNG", "PHẦN" followed by a space or colon and one or more digits (case-insensitive)
# The .*$ captures the rest of the line.
title_pattern = re.compile(r"^\s*([IVXLCDM]+\.\s+|(\d+\.)+\s+|[a-z][).]\s+|(bài|chương|phần)\s+\d+[:\s]+).*$", re.MULTILINE | re.IGNORECASE)

print("Regex pattern defined.")

Regex pattern defined.


In [3]:
for element in unstructured_elements:
    element_text = str(element)
    for i in element_text.split('\n'):
        if i:
            print('Line: ', i)
            print('-' * 10)
    print('-' * 40)

Line:  TÀI LIỆU TẬP HUẤN TOT VỀ KỸ THUẬT SẢN XUẤT HỒ TIÊU BỀN VỮNG
----------
Line:  1
----------
Line:  TÀI LIỆU TẬP HUẤN TOT VỀ KỸ THUẬT SẢN XUẤT HỒ TIÊU BỀN VỮNG
----------
Line:  3
----------
Line:  MỤC LỤC
----------
Line:  PHẦN I: HƯỚNG DẪN KỸ THUẬT SẢN XUẤT HỒ TIÊU
----------
Line:  BÀI 1: GIỐNG VÀ KỸ THUẬT TRỒNG MỚI, TÁI CANH HỒ TIÊU 1. Một số giống tiêu phổ biến ở Việt Nam 1.1. Giống tiêu Vĩnh Linh
----------
Line:  1.2. Giống Lada Belangtoeng
----------
Line:  1.3. Các giống tiêu sẻ
----------
Line:  1.4. Tiêu Ấn Độ
----------
Line:  1.5. Giống tiêu Trâu
----------
----------------------------------------
Line:  1.6. Giống Phú quốc
----------
Line:  2. Kỹ thuật nhân giống hồ tiêu: 2.1. Thiết kế vườn ươm giống tiêu 2.2. Chuẩn bị giá thể ươm
----------
Line:  2.3. Đóng bầu
----------
Line:  2.4. Chọn vườn tiêu lấy hom
----------
Line:  2.5. Xử lý hom tiêu
----------
Line:  2.6. Ươm, chăm sóc cây con
----------
Line:  2.7. Tiêu chuẩn cây tiêu giống trước khi xuất vườn.
--------

In [44]:
# Hàm phụ để kiểm tra xem dòng có phải là viết hoa toàn bộ không
def is_usable_data(line):
    words = line.split()
    if not words:
        return False

    # Nếu chỉ có 1 từ → Không ý nghĩa (bỏ page, BAI1, ...)
    if len(words) == 1:
        return False
    
    # Nếu viết hoa toàn bộ nhưng có từ viết thường
    for word in words:
        if not word.isupper():
            return False

    # Bỏ các dòng có dạng "Hình 10:", "Hinh 3.2 :", "HÌNH 5:" ...
    if re.match(r"^(HÌNH|HINH)\s*\d+([.:])", line, re.IGNORECASE):
        return False

    # Viết hoa toàn bộ (tối thiểu 2 ký tự)
    return True

In [45]:
all_lines = []
all_titles = []
all_titles_index = []

index = 0
for element in unstructured_elements:
    element_text = str(element)
    for i in element_text.split('\n'):
        if i.strip():
            all_lines.append(i.strip())

for index, value in enumerate(all_lines):
    for match in title_pattern.finditer(value):
        print('Regex Title: ', match.group(0).strip())
        print('-' * 10)
        all_titles_index.append(index)
        
    if is_usable_data(value):
        print('ALL CAPS Title: ', value)
        print('-' * 10)
        all_titles_index.append(index)


all_titles = [all_lines[i] for i in all_titles_index]

data_lines = {i: line for i, line in enumerate(all_lines)}
data_input = {}
for index, value in zip(all_titles_index, all_titles):
    data_input[index] = value

ALL CAPS Title:  TÀI LIỆU TẬP HUẤN TOT VỀ KỸ THUẬT SẢN XUẤT HỒ TIÊU BỀN VỮNG
----------
ALL CAPS Title:  TÀI LIỆU TẬP HUẤN TOT VỀ KỸ THUẬT SẢN XUẤT HỒ TIÊU BỀN VỮNG
----------
ALL CAPS Title:  MỤC LỤC
----------
ALL CAPS Title:  PHẦN I: HƯỚNG DẪN KỸ THUẬT SẢN XUẤT HỒ TIÊU
----------
Regex Title:  BÀI 1: GIỐNG VÀ KỸ THUẬT TRỒNG MỚI, TÁI CANH HỒ TIÊU 1. Một số giống tiêu phổ biến ở Việt Nam 1.1. Giống tiêu Vĩnh Linh
----------
Regex Title:  1.2. Giống Lada Belangtoeng
----------
Regex Title:  1.3. Các giống tiêu sẻ
----------
Regex Title:  1.4. Tiêu Ấn Độ
----------
Regex Title:  1.5. Giống tiêu Trâu
----------
Regex Title:  1.6. Giống Phú quốc
----------
Regex Title:  2. Kỹ thuật nhân giống hồ tiêu: 2.1. Thiết kế vườn ươm giống tiêu 2.2. Chuẩn bị giá thể ươm
----------
Regex Title:  2.3. Đóng bầu
----------
Regex Title:  2.4. Chọn vườn tiêu lấy hom
----------
Regex Title:  2.5. Xử lý hom tiêu
----------
Regex Title:  2.6. Ươm, chăm sóc cây con
----------
Regex Title:  2.7. Tiêu chuẩn c

In [46]:
data_input

{0: 'TÀI LIỆU TẬP HUẤN TOT VỀ KỸ THUẬT SẢN XUẤT HỒ TIÊU BỀN VỮNG',
 2: 'TÀI LIỆU TẬP HUẤN TOT VỀ KỸ THUẬT SẢN XUẤT HỒ TIÊU BỀN VỮNG',
 4: 'MỤC LỤC',
 5: 'PHẦN I: HƯỚNG DẪN KỸ THUẬT SẢN XUẤT HỒ TIÊU',
 6: 'BÀI 1: GIỐNG VÀ KỸ THUẬT TRỒNG MỚI, TÁI CANH HỒ TIÊU 1. Một số giống tiêu phổ biến ở Việt Nam 1.1. Giống tiêu Vĩnh Linh',
 7: '1.2. Giống Lada Belangtoeng',
 8: '1.3. Các giống tiêu sẻ',
 9: '1.4. Tiêu Ấn Độ',
 10: '1.5. Giống tiêu Trâu',
 11: '1.6. Giống Phú quốc',
 12: '2. Kỹ thuật nhân giống hồ tiêu: 2.1. Thiết kế vườn ươm giống tiêu 2.2. Chuẩn bị giá thể ươm',
 13: '2.3. Đóng bầu',
 14: '2.4. Chọn vườn tiêu lấy hom',
 15: '2.5. Xử lý hom tiêu',
 16: '2.6. Ươm, chăm sóc cây con',
 17: '2.7. Tiêu chuẩn cây tiêu giống trước khi xuất vườn.',
 18: '3. Kỹ thuật trồng mới hồ tiêu 3.1. Chọn đất trồng 3.2. Thiết kế lô, mật độ',
 19: '3.3. Các loại trụ trồng tiêu',
 20: '3.4. Thiết kế hệ thống cây chắn gió, che bóng vườn tiêu',
 21: '3.5. Xử lý đất và hố trước khi trồng',
 22: '3.6. Thời

In [43]:
text = 'Akm'
text.islower()

False

In [47]:
# Đây là toàn bộ tiêu đề mà tôi tách ra với regex (có thể bị mất hoặc bị thừa)
# Data đưa vào là dictionary với key là index của dòng tiêu đề, value là nội dung tiêu đề
# Cần xác định node con nhỏ nhất để lấy nội dung bên trong phạm vi tiêu đề đó đến trước tiêu đề tiếp theo
# Lúc gửi về thì theo dạng {'metadata': {index_1, index_2, ...}, 'content': [index_start, index_end]}
# metadata là tiêu đề phân cấp từ cao xuống thấp, cho tới tiêu đề của đoạn content, chỉ ghi index của nó ra. Còn content thì là đoạn văn bản bên trong phạm vi tiêu đề đó, index_start là từ index bắt đầu, còn index_end là trước khi đến tiêu đề tiếp theo (nhận biết đến đó là hết content)
# Đoạn nào không có ý nghĩa thì bỏ qua.

# PROMPT
prompt = '''
# Nhiệm vụ
Bạn được cung cấp một dictionary chứa các tiêu đề đã được tách ra từ một tài liệu.
Key là chỉ số dòng (index) trong tài liệu, value là nội dung tiêu đề tương ứng.

Nhiệm vụ của bạn là:
1. Xác định cấu trúc phân cấp của các tiêu đề (cha – con – cháu).
2. Xác định đoạn nội dung thuộc về mỗi tiêu đề nhỏ nhất (node con cuối cùng).

# Đầu vào
- Một dictionary dạng {index: "text"}, trong đó index là số dòng, text là nội dung tiêu đề.
- Các tiêu đề có thể bị thiếu, thừa hoặc không đồng nhất về format (VD: “Chương 1”, “1.1”, “I.”, “A.”, “Phần 2”, “Bài 3”, …).

# Quy tắc phân tích
- Xác định cấp độ tiêu đề dựa vào pattern (VD: "Chương" > "Phần" > "Bài").
- Nếu không xác định được rõ cấp độ, suy luận theo logic vị trí (index nhỏ hơn thường là cấp cao hơn).
- Các tiêu đề không mang ý nghĩa rõ ràng hoặc trùng lặp (VD: “BÀI”, “PHẦN”, “KẾT LUẬN”) thì bỏ qua.
- Mỗi node con nhỏ nhất cần được xác định phạm vi nội dung của nó:
  - `index_start`: dòng ngay sau tiêu đề hiện tại.
  - `index_end`: dòng trước khi đến tiêu đề tiếp theo (hoặc kết thúc tài liệu).

# Đầu ra
Trả về danh sách các đối tượng, mỗi đối tượng có dạng:

[
  [index_cấp_1, index_cấp_2, ..., index_cấp_n],
  [index_start, index_end]
]

Trong đó:
- (index 0 trong đối tượng) là mảng chứa các chỉ số index tiêu đề từ cao nhất → thấp nhất → tiêu đề hiện tại.
- (index 1 trong đối tượng) xác định phạm vi văn bản nằm trong tiêu đề cuối cùng đó.

# Ghi chú
- Bỏ qua các đoạn không có nội dung hoặc chỉ chứa tiêu đề rỗng.
- Không được lặp lại phạm vi trùng nhau giữa các node.
- Phải đảm bảo logic cha–con nhất quán (nội dung con nằm hoàn toàn trong phạm vi cha).
- Viết liền trên 1 đoạn, không xuống dòng hay tab.

# Ví dụ
Giả sử có dữ liệu:
{
  1: "CHƯƠNG 1: GIỚI THIỆU",
  5: "1.1 Tổng quan",
  9: "1.1.1 Mục tiêu nghiên cứu",
  15: "CHƯƠNG 2: CƠ SỞ LÝ THUYẾT",
  16: "CHƯƠNG 3: CƠ SỞ THỰC HÀNH",
  ...
}

Thì đầu ra mong đợi như dưới, lấy cả nội dung từ dòng 9 đến 14 (trước khi đến dòng 15): 
[[[1, 5, 9],[9, 14]], [[15],[15, 15]],...]
'''

In [48]:
import dotenv
dotenv.load_dotenv()

from llms_service import get_llm_service, ResponseTitleSchema
from prompts import SYSTEM_CONTENT_REFACTOR_TITLES

In [49]:
llms = get_llm_service("gemini-2.5-flash")

In [50]:
response = llms.invoke(prompt, str(data_input))

In [55]:
response

'[\n  {\n    "metadata": [\n      5\n    ],\n    "conntent": [\n      5,\n      5\n    ]\n  },\n  {\n    "metadata": [\n      5,\n      6\n    ],\n    "conntent": [\n      6,\n      6\n    ]\n  },\n  {\n    "metadata": [\n      5,\n      6,\n      7\n    ],\n    "conntent": [\n      7,\n      7\n    ]\n  },\n  {\n    "metadata": [\n      5,\n      6,\n      8\n    ],\n    "conntent": [\n      8,\n      8\n    ]\n  },\n  {\n    "metadata": [\n      5,\n      6,\n      9\n    ],\n    "conntent": [\n      9,\n      9\n    ]\n  },\n  {\n    "metadata": [\n      5,\n      6,\n      10\n    ],\n    "conntent": [\n      10,\n      10\n    ]\n  },\n  {\n    "metadata": [\n      5,\n      6,\n      11\n    ],\n    "conntent": [\n      11,\n      11\n    ]\n  },\n  {\n    "metadata": [\n      5,\n      6,\n      12\n    ],\n    "conntent": [\n      12,\n      12\n    ]\n  },\n  {\n    "metadata": [\n      5,\n      6,\n      12,\n      13\n    ],\n    "conntent": [\n      13,\n      13\n    ]\n 

In [19]:
response_text = ""
for chunk in response.iter_content(chunk_size=None, decode_unicode=True):
    response_text += chunk

AttributeError: 'str' object has no attribute 'iter_content'

In [52]:
import json
with open("debug.json", "w", encoding="utf-8") as f:
    json.dump(response, f, ensure_ascii=False, indent=2)

In [56]:
import json

def read_json_response(response):
    
    # start_index = response.find('```json') + len('```json')
    # end_index = response.rfind('```')
    # json_string = response[start_index:end_index].strip()

    # Parse the JSON string into a Python object (list of dictionaries)
    # structured_data = json.loads(json_string)

    structured_data = json.loads(response)
    return structured_data

In [57]:
response_list_index = read_json_response(response)
response_list_index

[{'metadata': [5], 'conntent': [5, 5]},
 {'metadata': [5, 6], 'conntent': [6, 6]},
 {'metadata': [5, 6, 7], 'conntent': [7, 7]},
 {'metadata': [5, 6, 8], 'conntent': [8, 8]},
 {'metadata': [5, 6, 9], 'conntent': [9, 9]},
 {'metadata': [5, 6, 10], 'conntent': [10, 10]},
 {'metadata': [5, 6, 11], 'conntent': [11, 11]},
 {'metadata': [5, 6, 12], 'conntent': [12, 12]},
 {'metadata': [5, 6, 12, 13], 'conntent': [13, 13]},
 {'metadata': [5, 6, 12, 14], 'conntent': [14, 14]},
 {'metadata': [5, 6, 12, 15], 'conntent': [15, 15]},
 {'metadata': [5, 6, 12, 16], 'conntent': [16, 16]},
 {'metadata': [5, 6, 12, 17], 'conntent': [17, 17]},
 {'metadata': [5, 6, 12, 18], 'conntent': [18, 18]},
 {'metadata': [5, 6, 12, 18, 19], 'conntent': [19, 19]},
 {'metadata': [5, 6, 12, 18, 20], 'conntent': [20, 20]},
 {'metadata': [5, 6, 12, 18, 21], 'conntent': [21, 21]},
 {'metadata': [5, 6, 12, 18, 22], 'conntent': [22, 46]},
 {'metadata': [5, 6, 12, 18, 47], 'conntent': [47, 47]},
 {'metadata': [5, 6, 12, 48],

In [58]:
def convert_index_to_text(structured_data, data_lines):
    results = []
    for block in structured_data:
        metadata_texts = [data_lines.get(idx, f"[MISSING {idx}]") for idx in block.get('metadata', [])]
        start, end = block.get('conntent', [None, None]) # ?????????????
        
        if start is not None and end is not None:
            # Lấy toàn bộ text trong range [start, end]
            content_texts = [data_lines.get(i, f"[MISSING {i}]") for i in range(start, end)]
        else:
            content_texts = []

        results.append({
            "metadata": metadata_texts,
            "content": content_texts
        })
    return results

In [59]:
response_list_text = convert_index_to_text(response_list_index, data_lines)
response_list_text

[{'metadata': ['PHẦN I: HƯỚNG DẪN KỸ THUẬT SẢN XUẤT HỒ TIÊU'], 'content': []},
 {'metadata': ['PHẦN I: HƯỚNG DẪN KỸ THUẬT SẢN XUẤT HỒ TIÊU',
   'BÀI 1: GIỐNG VÀ KỸ THUẬT TRỒNG MỚI, TÁI CANH HỒ TIÊU 1. Một số giống tiêu phổ biến ở Việt Nam 1.1. Giống tiêu Vĩnh Linh'],
  'content': []},
 {'metadata': ['PHẦN I: HƯỚNG DẪN KỸ THUẬT SẢN XUẤT HỒ TIÊU',
   'BÀI 1: GIỐNG VÀ KỸ THUẬT TRỒNG MỚI, TÁI CANH HỒ TIÊU 1. Một số giống tiêu phổ biến ở Việt Nam 1.1. Giống tiêu Vĩnh Linh',
   '1.2. Giống Lada Belangtoeng'],
  'content': []},
 {'metadata': ['PHẦN I: HƯỚNG DẪN KỸ THUẬT SẢN XUẤT HỒ TIÊU',
   'BÀI 1: GIỐNG VÀ KỸ THUẬT TRỒNG MỚI, TÁI CANH HỒ TIÊU 1. Một số giống tiêu phổ biến ở Việt Nam 1.1. Giống tiêu Vĩnh Linh',
   '1.3. Các giống tiêu sẻ'],
  'content': []},
 {'metadata': ['PHẦN I: HƯỚNG DẪN KỸ THUẬT SẢN XUẤT HỒ TIÊU',
   'BÀI 1: GIỐNG VÀ KỸ THUẬT TRỒNG MỚI, TÁI CANH HỒ TIÊU 1. Một số giống tiêu phổ biến ở Việt Nam 1.1. Giống tiêu Vĩnh Linh',
   '1.4. Tiêu Ấn Độ'],
  'content': []},
 {'metad

In [60]:
with open("test.json", "w", encoding="utf-8") as f:
    json.dump(response_list_text, f, ensure_ascii=False, indent=2)

In [61]:
data = []
with open("test.json", "r", encoding="utf-8") as f:
    data = json.load(f)

data

[{'metadata': ['PHẦN I: HƯỚNG DẪN KỸ THUẬT SẢN XUẤT HỒ TIÊU'], 'content': []},
 {'metadata': ['PHẦN I: HƯỚNG DẪN KỸ THUẬT SẢN XUẤT HỒ TIÊU',
   'BÀI 1: GIỐNG VÀ KỸ THUẬT TRỒNG MỚI, TÁI CANH HỒ TIÊU 1. Một số giống tiêu phổ biến ở Việt Nam 1.1. Giống tiêu Vĩnh Linh'],
  'content': []},
 {'metadata': ['PHẦN I: HƯỚNG DẪN KỸ THUẬT SẢN XUẤT HỒ TIÊU',
   'BÀI 1: GIỐNG VÀ KỸ THUẬT TRỒNG MỚI, TÁI CANH HỒ TIÊU 1. Một số giống tiêu phổ biến ở Việt Nam 1.1. Giống tiêu Vĩnh Linh',
   '1.2. Giống Lada Belangtoeng'],
  'content': []},
 {'metadata': ['PHẦN I: HƯỚNG DẪN KỸ THUẬT SẢN XUẤT HỒ TIÊU',
   'BÀI 1: GIỐNG VÀ KỸ THUẬT TRỒNG MỚI, TÁI CANH HỒ TIÊU 1. Một số giống tiêu phổ biến ở Việt Nam 1.1. Giống tiêu Vĩnh Linh',
   '1.3. Các giống tiêu sẻ'],
  'content': []},
 {'metadata': ['PHẦN I: HƯỚNG DẪN KỸ THUẬT SẢN XUẤT HỒ TIÊU',
   'BÀI 1: GIỐNG VÀ KỸ THUẬT TRỒNG MỚI, TÁI CANH HỒ TIÊU 1. Một số giống tiêu phổ biến ở Việt Nam 1.1. Giống tiêu Vĩnh Linh',
   '1.4. Tiêu Ấn Độ'],
  'content': []},
 {'metad

In [62]:
data_filtered = []
for item in data:
    if item.get('content', []):
        data_filtered.append(item)

In [63]:
with open("test_next.json", "w", encoding="utf-8") as f:
    json.dump(data_filtered, f, ensure_ascii=False, indent=2)